In [1]:

#Need a remake# 

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
from torchvision import transforms
import time
from tqdm.autonotebook import tqdm
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from inspect import getfullargspec,signature

import matplotlib.pyplot as plt
import numpy as np

from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import datasets


%matplotlib inline
%config InlineBackend.figure_format = 'svg'


In [2]:
###THis file is to generate standard model ############

In [3]:
####################################
#Need to run the same with FashionMNIST
###################################


############################
class CinicResNet(nn.Module):
  def __init__(self, in_channels=3):
    super(CinicResNet, self).__init__()

    # Load a pretrained resnet model from torchvision.models in Pytorch
    self.model = models.resnet50(pretrained=False)

    # Change the input layer to take Grayscale image, instead of RGB images.  **************
    # Hence in_channels is set as 1 or 3 respectively
    # original definition of the first layer on the ResNet class
    # self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
    self.model.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
    
    # Change the output layer to output 10 classes instead of 1000 classes
    num_ftrs = self.model.fc.in_features
    self.model.fc = nn.Linear(num_ftrs, 10)

  def forward(self, x):
    return self.model(x)


my_resnet = CinicResNet()

input = torch.randn((16,3,244,244))
output = my_resnet(input)
print(output.shape)

print(my_resnet)

torch.Size([16, 10])
CinicResNet(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample):

In [4]:
import torch
import torch.nn as nn
from torchvision.models import resnet18

from robustness.datasets import CINIC
ds = CINIC('/tmp/cinic')


def calculate_metric(metric_fn, true_y, pred_y):
    if "average" in signature(metric_fn).parameters:
        return metric_fn(true_y, pred_y, average="micro")
    else:
        return metric_fn(true_y, pred_y)
    
def print_scores(p, r, f1, a, batch_size):
    for name, scores in zip(("precision", "recall", "F1", "accuracy"), (p, r, f1, a)):
        print(f"\t{name.rjust(14, ' ')}: {sum(scores)/batch_size:.4f}")
        
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

In [5]:
print(torch.cuda.is_available())

True


In [6]:
# values are standard normalization for ImageNet images, 
# from https://github.com/pytorch/examples/blob/master/imagenet/main.py
#norm = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

# load pre-trained ResNet18, and put into evaluation mode (necessary to e.g. turn off batchnorm)
#model = resnet50(pretrained=True)
#model.eval();
# model:
model = CinicResNet().to(device)

# params you need to specify:
epochs = 200
NUM_WORKERS = 8
BATCH_SIZE = 128

# Dataloaders
train_loader, val_loader = ds.make_loaders(workers=NUM_WORKERS, batch_size=BATCH_SIZE)

# loss function and optimiyer
loss_function = nn.CrossEntropyLoss() # your loss function, cross entropy works well for multi-class problems

# optimizer, I've used Adadelta, as it wokrs well without any magic numbers
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4) # Using Karpathy's learning rate constant

start_ts = time.time()

losses = []
batches = len(train_loader)
val_batches = len(val_loader)
print(batches,val_batches)

==> Preparing dataset cinic..
704 704


In [7]:
# IF PRETRAINED  SKIP THIS SECTION

# loop for every epoch (training + evaluation)
for epoch in range(epochs):
    total_loss = 0

    # progress bar (works in Jupyter notebook too!)
    progress = tqdm(enumerate(train_loader), desc="Loss: ", total=batches)

    # ----------------- TRAINING  -------------------- 
    # set model to training
    model.train()
    
    for i, data in progress:
        X, y = data[0].to(device), data[1].to(device)
        
        # training step for single batch
        model.zero_grad()
        outputs = model(X)
        loss = loss_function(outputs, y)
        loss.backward()
        optimizer.step()

        # getting training quality data
        current_loss = loss.item()
        total_loss += current_loss

        # updating progress bar
        progress.set_description("Loss: {:.4f}".format(total_loss/(i+1)))
        
    # releasing unceseccary memory in GPU
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    # ----------------- VALIDATION  ----------------- 
    val_losses = 0
    precision, recall, f1, accuracy = [], [], [], []
    
    # set model to evaluating (testing)
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            X, y = data[0].to(device), data[1].to(device)

            outputs = model(X) # this get's the prediction from the network

            val_losses += loss_function(outputs, y)

            predicted_classes = torch.max(outputs, 1)[1] # get class from network's prediction
            
            # calculate P/R/F1/A metrics for batch
            for acc, metric in zip((precision, recall, accuracy), 
                                   (precision_score, recall_score, accuracy_score)):
                acc.append(
                    calculate_metric(metric, y.cpu(), predicted_classes.cpu())
                )
          
    print(f"Epoch {epoch+1}/{epochs}, training loss: {total_loss/batches}, validation loss: {val_losses/val_batches}")
    print_scores(precision, recall, f1, accuracy, val_batches)
    losses.append(total_loss/batches) # for plotting learning curve
print(f"Training time: {time.time()-start_ts}s")

            


Loss:   0%|          | 0/704 [00:00<?, ?it/s]

Epoch 1/200, training loss: 1.9731782879680395, validation loss: 2.7014806270599365
	     precision: 0.3501
	        recall: 0.3501
	            F1: 0.0000
	      accuracy: 0.3501


Loss:   0%|          | 0/704 [00:00<?, ?it/s]

Epoch 2/200, training loss: 1.7542736496437679, validation loss: 2.3254733085632324
	     precision: 0.3995
	        recall: 0.3995
	            F1: 0.0000
	      accuracy: 0.3995


Loss:   0%|          | 0/704 [00:00<?, ?it/s]

...

Epoch 198/200, training loss: 0.19202800609425388, validation loss: 1.2094365358352661
	     precision: 0.7343
	        recall: 0.7343
	            F1: 0.0000
	      accuracy: 0.7343


Loss:   0%|          | 0/704 [00:00<?, ?it/s]

Epoch 199/200, training loss: 0.18260254253717986, validation loss: 1.2888765335083008
	     precision: 0.7307
	        recall: 0.7307
	            F1: 0.0000
	      accuracy: 0.7307


Loss:   0%|          | 0/704 [00:00<?, ?it/s]

Epoch 200/200, training loss: 0.1838479452523064, validation loss: 1.3499754667282104
	     precision: 0.7267
	        recall: 0.7267
	            F1: 0.0000
	      accuracy: 0.7267
Training time: 15537.662685155869s


In [8]:
torch.save(model.state_dict(), "./CinicSTD50Net")    
model = CinicResNet()
model_state_dict = torch.load("./CinicSTD50Net")
model.load_state_dict(model_state_dict)

<All keys matched successfully>